In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
from bs4 import BeautifulSoup as bs
import requests
import csv

os.path.dirname(sys.executable)
from time import sleep
from selenium import webdriver

In [3]:
key_words = ["Narco+pista", "Narcopista", "Pista+Clandestina", "Narco+avioneta", "Narco+ganad", "Narco+ganadería", "Narco+ganadero", "Narco+ganado", "Narco+Incauta", "narco+incautado", "narco+incuatan", "Narco+Soborno+o+Coima+Droga", "Narco+lancha", "Narco+playa", "Narco+barco", "Narco+submarino", "Narco+lavado", "Narco+minería", "Narco+vida+silvestre", "Narco+Palma+Africana", "Narco+Incauta", "Cocaína+decomisada" , "Cocaína+incauta", "Narco+lavado" , "Narco+Blanqueo" , "Narco+Legitimación+de+capitales", "Narco+Bodega", "Narco+Mar" , "Narco+Océano", "Narco+Costa", "Narco+Puerto", "Pesca+con+narco+o+cocaína"]


In [4]:
def get_search_urls_by_year(year):
    search_urls=[]
    for key in key_words:
        alt_string=''
        key_list=key.split('+')
        for i in range(0, len(key_list)):
            if i==0:
                alt_string+=key_list[i]
            else:
                alt_string+="%20"+key_list[i]
        alt_string+="%20"+str(year)
        for page in range(1,6):
            url="https://www.laprensa.hn/buscador/?q="+key+"+"+str(year)+"#gsc.tab=0&gsc.q="+alt_string+"&gsc.page="+str(page)
            url_tuple = (url,key)
            #print(url)
            search_urls.append(url_tuple)
    return search_urls

In [5]:
search_urls_2017 = get_search_urls_by_year(2017)
search_urls_2016 = get_search_urls_by_year(2016)
search_urls_2015 = get_search_urls_by_year(2015)

In [11]:
def get_article_links(search_url):
    browser = webdriver.Firefox()
    links=[]
    try:
        browser.get(search_url)
    except:
        return links
    a_list = browser.find_elements_by_class_name("gs-title")
    for a in a_list:
        l=a.get_attribute('href')
        if l not in links:
            if l is not None and l is not '':
                links.append(l)
    #print(links)
    browser.close()
    return links

In [16]:
articles_2017 = get_article_links(search_urls_2017[0])

['https://www.laprensa.hn/videos/?mId=XtRUrHaL&c=US', 'https://www.laprensa.hn/honduras/1307234-410/destruyen-honduras-26-pista-clandestina-usada-narcotrafico-2019', 'https://www.laprensa.hn/sucesos/1328586-410/honduras-destruyen-pista-clandestina-narcotrafico-', 'https://www.laprensa.hn/sucesos/1158025-410/narcopista-trafico_droga-pista_clandestina-honduras-', 'https://www.laprensa.hn/honduras/1118994-410/seis-a%C3%B1os-de-lucha-contra-el-narco', 'https://www.laprensa.hn/sucesos/1138216-410/decomisados-kilos-coca-narcotrafico-marihuana', 'https://www.laprensa.hn/honduras/1051562-410/testigo-asegura-que-pista-en-col%C3%B3n-fue-usada-por-narcos', 'https://www.laprensa.hn/honduras/1035192-410/fusina-inhabilita-otra-narcopista-en-puerto-cort%C3%A9s', 'https://www.laprensa.hn/honduras/1149513-410/destruccion-pista-narcotrafico-honduras-gracias_a_dios', 'https://www.laprensa.hn/honduras/1051836-410/caso-fabio-lobo-cachiro-tambi%C3%A9n-implica-a-hermano-de-mel-zelaya']


In [14]:
def scrape_articles_to_text_file(articles, num, year,term):
    batch = []
    for paper in articles:
        if paper=='' or paper==None:
            continue
        if "video" in paper:
            continue
        #try:
        #    payload = {'api_key': '1b5ed767ba936e3b00cffc923487cdad', 'url': link}
        #    page = requests.get("http://api.scraperapi.com",params=payload, timeout=30)
        #except:
        #    print('connection failed')
        #    continue
        paper_dict = {}
        paper_dict['year'] = year
        paper_dict['keyword'] = term
        try:
            page = requests.get(paper)
        except:
            continue
        soup = bs(page.content, 'html.parser')
        #filename="Corpus/Honduras/neg/hn_neg"+str(num+1)+".txt"
        #file=open(filename,"w",encoding='utf-8')
        h1=soup.find_all('h1')
        try:
            title=h1[0].text
            #print(title)
            #file.write(title)
            #file.write("\n")
            paper_dict['title'] = title
        except:
            #print("no title")
            #file.write("")
            paper_dict['title'] = ""

        pAll = soup.find_all('p')
        all_text = ''
        for p in pAll:
            #file.write(p.text)
            all_text+=p.text
        paper_dict['article'] = all_text
        #file.close()
        batch.append(paper_dict)
        num=num+1
    return batch, num

In [21]:
count = scrape_articles_to_text_file(articles_2017,0)

Destruyen en Honduras la 26 pista clandestina usada para narcotráfico en 2019
Honduras destruyen 32 pista clandestina usada para narcotráfico
Destruyen pista clandestina usada para narcotráfico
Seis años de lucha contra el narco
Decomisados más de 1,700 kilos de coca al narcotráfico en 2017
Testigo asegura que pista en Colón fue usada por narcos
Fusina inhabilita otra narcopista en Puerto Cortés
Destruyen pista clandestina usada para narcotráfico en Gracias a Dios
Caso Fabio Lobo: Cachiro también implica a hermano de Mel Zelaya


In [28]:
#scrape all 2017
search_urls_2017 = get_search_urls_by_year(2017)
articles_2017=[]
for search_url in search_urls_2017:
    articles_2017+=get_article_links(search_url)
count2017 = scrape_articles_to_text_file(articles_2017,0)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [5]:
#scrape all 2016
search_urls_2016 = get_search_urls_by_year(2016)
articles_2016=[]
count = count2017
for search_url in search_urls_2016:
    articles = get_article_links(search_url)
    articles_2016+=articles
    count = scrape_articles_to_text_file(articles,count)
count2016 = count

NameError: name 'key_words' is not defined

In [12]:
#scrape all 2015
search_urls_2015 = get_search_urls_by_year(2015)
articles_2015=[]
count = 1799
for search_url in search_urls_2015:
    articles = get_article_links(search_url)
    articles_2015+=articles
    count = scrape_articles_to_text_file(articles,count)

In [15]:
#4-20
master_list = []
count = 0
for year in range(2017,2006,-1):
    search_urls = get_search_urls_by_year(year)
    for url in search_urls:
        articles = get_article_links(url[0])
        mini_list, count = scrape_articles_to_text_file(articles,count,year,url[1])
        master_list+=mini_list
        print(len(master_list))

9
17
25
34
43
50
57
66
76
84
94
104
113
121


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


130
140
149
158
167
177
186
195
204
213
221
231
241
251
260
270
280
290
299
303
303
313
323
332
342
352
361
371
381
391
400
410
420
430
440
450
458
468
477
487
496
506
513
513
513
513
523
533
543
552
561
571
581
590
600
610
619
629
639
649
658
667
676
686
696
706
715
725
735
744
754
764
772
776
776
776
784
784
784
784
784
794


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


797
797
797
797
806
816
826
836
845
855
865
875
885
894
904
912
922
932
940
949
949
959
968
978
988
998
1000
1000
1000
1007
1007
1007
1007
1007
1017
1024
1034
1042
1042
1052
1062
1072
1081
1091
1101
1111
1121
1129
1137
1147
1156
1166
1174
1181
1191
1201
1210
1219
1229
1234
1234
1234
1234
1234
1244
1252
1261
1270
1280
1288
1298
1308
1310
1310
1320
1330
1340
1349
1359
1369
1379
1389
1399
1399
1409
1419
1429
1439
1449
1459
1469
1479
1486
1486
1496
1506
1508
1508
1508
1518
1528
1538
1548
1558
1568
1578
1588
1598
1607
1617
1627
1636
1646
1656
1666
1676
1686
1696
1706
1714
1714
1714
1714
1714
1723
1733
1743
1753
1763
1773
1783
1793
1803
1813
1823
1832
1842
1852
1862
1872
1882
1892
1902
1906
1916
1926
1936
1946
1956
1966
1976
1984
1984
1984
1988
1988
1988
1988
1988
1998
2008
2008
2008
2008
2018
2028
2034
2044
2053
2063
2073
2083
2093
2103
2113
2123
2132
2140
2149
2159
2169
2179
2189
2199
2209
2219
2224
2224
2224
2226
2226
2226
2226
2226
2236
2246
2256
2259
2259
2269
2279
2289
2299
2308
2317
2

...

8105
8112
8122
8132
8132
8142
8152
8162
8172
8182
8192
8202
8212
8222
8232
8242
8252
8262
8272
8282
8292
8302
8312
8317
8317
8317
8317
8322
8322
8322
8322
8322
8326
8326
8326
8326
8326
8336
8346
8356
8366


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


8376
8386
8396
8406
8416
8426
8436
8445
8454
8464
8473
8483
8493
8503
8513
8523
8533
8541
8541
8541
8541
8549
8549
8549
8549
8549
8559
8569
8577
8577
8577
8587
8596
8606
8616
8626
8636
8646
8656
8658
8658
8668
8678
8688
8698
8708
8718
8728
8738
8748
8758
8765
8765
8765
8765
8765
8774
8784
8793
8802
8812
8821
8831
8841
8847
8847
8857
8867
8877
8887
8897
8906
8916
8926
8936
8946
8956
8966
8976
8986
8995
9005
9015
9015
9025
9032
9042
9052
9061
9061
9061
9071
9081
9090
9100
9110
9120
9130
9140
9150
9159
9169
9179
9189
9199
9209
9219
9229
9239
9249
9259
9268
9272
9272
9272
9272
9282
9292
9301
9310
9320
9330
9340
9349
9359
9369
9379
9389
9399
9409
9418
9428
9432
9432
9432
9432
9442
9451
9461
9471
9481
9491
9501
9510
9520
9526
9530
9530
9530
9530
9530
9540
9547
9547
9547
9547
9557
9567
9577
9587
9596
9606
9616
9626
9636
9646
9656
9666
9676
9686
9696
9706
9715
9725
9735
9745
9754
9764
9766
9766
9766
9772
9772
9772
9772
9772
9782
9792
9802
9805
9805
9815
9823
9833
9833
9843
9853
9863
9872
9872


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


12311
12311
12311
12321
12331
12340
12350
12360
12370
12380
12390
12400
12410
12420
12430
12432
12432
12432
12442
12448
12448
12448
12448
12458
12468
12478
12488
12498
12508
12508
12518
12527
12527
12537
12547
12557
12567
12577
12585
12595
12605
12614
12623
12630
12630
12630
12630
12630
12640
12650
12657
12657
12657
12667
12677
12687
12697
12707
12717
12727
12737
12745
12745
12755
12756
12756
12756
12756
12766
12776
12786
12796
12806
12816
12819
12819
12819
12819
12828
12828
12828
12828
12828
12838
12838
12838
12838
12838
12848
12858
12868
12877
12877
12887
12897
12907
12917
12927
12937
12947
12957
12967
12977
12987
12997
13007
13017
13027
13037
13040
13040
13040
13040
13041
13041
13041
13041
13041
13051
13052
13052
13052
13052
13062
13072
13082
13092
13102
13112
13122
13122
13122
13122
13132
13142
13152
13162
13172
13182
13192
13202
13212
13222
13222
13222
13222
13222
13222


In [23]:
with open('honduras_neg.csv', 'w',encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, ['year','keyword','title','article'])
                writer.writeheader()
                for data in master_list:
                    writer.writerow(data)

{'year': 2009,
 'keyword': 'Narco+ganado',
 'title': 'Gobierno confirma detención de Yankel Rosenthal en Miami',
 'article': 'El empresario sampedrano será presentado el miércoles ante tribunales estadounidenses.Tegucigalpa, Honduras.El Gobierno de Honduras emitió un comunicado confirmando la detención del empresario sampedrano y dirigente de fútbol, Yankel Rosenthal.El exministro de Inversión de este Gobierno fue detenido la tarde del martes en el aeropuerto internacional de Miami, Estados Unidos.Rosenthal salió de Honduras en horas de la tarde desde San Pedro Sula y arribó a eso de las 6:00 pm (hora local) a Miami, donde fue abordado por autoridades de migración estadounidenses."Esta noche el Gobierno de la República de Honduras fue informado por la Misión Diplomática de los Estados Unidos de América en Tegucigalpa, de la captura el día de hoy (martes) del ciudadano hondureño Yankel Rosenthal. El señor Rosenthal será presentado el día de mañana 7 de octubre ante los Tribunales de Jus